# T0. PREPARE POINT CLOUDS

Import and prepare CVPR point clouds for t1-t10.
To run these scripts, create a python 3.10 environment & install geomapi (numpy, opend3d, ifcopenshell, trimesh, ...)

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import time
import json
from scipy.spatial.transform import Rotation   
import copy
from datetime import datetime

import geomapi
from geomapi.nodes import *
from geomapi import utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.tools import progresstools as pt

import geomapi.tools as tl

import context
import utils as utl
import utils.t0_utils as t0


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
start_time = time.time()

In [5]:
path=Path(os.getcwd()).parents[2]/'data'
input_folder=path / 't0'/'train'
class_file=path/'_classes.json'
output_folder= path/'t0'/'results'
output_folder.mkdir(parents=True, exist_ok=True)
graphPath=str(path/f'objectGraph.ttl')

distance_threshold=0.1
resolution=0.03

import classes

In [6]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


import objects

In [7]:
files=[f for f in ut.get_list_of_files(input_folder) if f.endswith('.json')]
objects_dict = {}  
objectNodes=[]
for file in files:
    # Load the JSON data into Python objects
    t0.parse_json(file,objects_dict)

# Print counts
print(f'len columns: {sum(1 for item in objects_dict.values() if item["type"] == "columns")}')
print(f'len doors: {sum(1 for item in objects_dict.values() if item["type"] == "doors")}')
print(f'len walls: {sum(1 for item in objects_dict.values() if item["type"] == "walls")}')

#convert json to object nodes
objectNodes.extend(t0.create_object_nodes(objects_dict,class_dict))

print(f'len objectNodes: {len(objectNodes)}')

Data read from file: 05_MedOffice_01_F2_columns
Data read from file: 05_MedOffice_01_F2_doors
Data read from file: 05_MedOffice_01_F2_walls
Data read from file: 06_MedOffice_02_B1_columns
Data read from file: 06_MedOffice_02_B1_doors
Data read from file: 06_MedOffice_02_B1_walls
Data read from file: 06_MedOffice_02_F1_columns
Data read from file: 06_MedOffice_02_F1_doors
Data read from file: 06_MedOffice_02_F1_walls
Data read from file: 06_MedOffice_02_F2_columns
Data read from file: 06_MedOffice_02_F2_doors
Data read from file: 06_MedOffice_02_F2_walls
Data read from file: 06_MedOffice_02_F3_columns
Data read from file: 06_MedOffice_02_F3_doors
Data read from file: 06_MedOffice_02_F3_walls
Data read from file: 07_MedOffice_03_F3_columns
Data read from file: 07_MedOffice_03_F3_doors
Data read from file: 07_MedOffice_03_F3_walls
Data read from file: 07_MedOffice_03_F4_columns
Data read from file: 07_MedOffice_03_F4_doors
Data read from file: 07_MedOffice_03_F4_walls
Data read from file:

Import PCD

In [8]:
files=[f for f in ut.get_list_of_files(input_folder) if f.endswith('.laz')]

#iterate over files
for f in files: 
    
    #read pcd
    print(f)
    las=laspy.read(f)
    
    #create nodes
    
    pcdNode=PointCloudNode(resource=gmu.las_to_pcd(las),
                                    name='_'.join(f.split('/')[-1].split('.')[0].split('_')[0:-1]),
                                    color=ut.random_color()) 
    
      
    class_scalar,object_scalar=t0.process_point_cloud(pcdNode,
                                                      [n for n in objectNodes if n.derivedFrom==pcdNode.name],
                                                      distance_threshold=distance_threshold,
                                                      resolution=resolution)
    
    gmu.las_add_extra_dimensions(las,(class_scalar,object_scalar),['classes','objects'],['uint8','uint16'])
    
    las.write(output_folder/f'{pcdNode.name}_pcd.laz')
    print(f'{pcdNode.name}_pcd.laz')
    print(f'Elapsed time: {time.time() - start_time}')

c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/05_MedOffice_01_F2_s0p01m.laz
05_MedOffice_01_F2_pcd.laz
Elapsed time: 201.8166539669037
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_B1_s0p01m.laz
06_MedOffice_02_B1_pcd.laz
Elapsed time: 1159.0312247276306
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_F1_s0p01m.laz
06_MedOffice_02_F1_pcd.laz
Elapsed time: 2279.401542901993
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_F2_s0p01m.laz
06_MedOffice_02_F2_pcd.laz
Elapsed time: 3060.8463344573975
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_F3_s0p01m.laz
06_MedOffice_02_F3_pcd.laz
Elapsed time: 3716.6023156642914
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BI

In [9]:
# joined_geometries=gmu.join_geometries([n.resource for n in objectNodes if n.derivedFrom==pcdNode.name])
# lines=gmu.join_geometries([n.line.paint_uniform_color(n.color) for n in objectNodes if n.derivedFrom==pcdNode.name 
#                            if getattr(n,'line',None) is not None])

# o3d.visualization.draw_geometries([joined_geometries,lines])

In [10]:
# o=0
# walls=gmu.join_geometries([n.resource.paint_uniform_color([1,0,0]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                            if n.class_name in ['walls']])
# columns=gmu.join_geometries([n.resource.paint_uniform_color([0,1,0]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                              if n.class_name in ['columns']])
# doors=gmu.join_geometries([n.resource.paint_uniform_color([0,0,1]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                             if n.class_name in ['doors']])
# joined_geometries=gmu.join_geometries([walls,columns,doors])
# print(joined_geometries)

# lines=gmu.join_geometries([n.line.paint_uniform_color(n.color) for n in objectNodes if n.derivedFrom==pcdNodes[o].name 
#                            if getattr(n,'line',None) is not None])

# o3d.visualization.draw_geometries([pcdNodes[o].resource] + [joined_geometries,lines])

## PROCESSING

In [11]:
# objectNodes= [n for n in objectNodes if n.derivedFrom==pcdNode.name]

In [12]:
# #create an identity point cloud of all the objectNodes
# identityPcd,objectArray=gmu.create_identity_point_cloud([n.resource for n in objectNodes if n.derivedFrom==pcdNode.name],resolution=0.05)
# classArray=np.array([int(n.class_id) for n in objectNodes if n.derivedFrom==pcdNode.name])[objectArray.astype(int)]
# print(len(classArray),len(objectArray))

In [13]:
# #compute distances
# distances_total=pcdNodes[o].resource.compute_point_cloud_distance(identityPcd)
# distances_total=np.asarray(distances_total)

In [14]:
# object_scalar = np.full(len(pcdNode.resource.points), 0, dtype=np.uint8)
# class_scalar = np.full(len(pcdNode.resource.points), 255, dtype=np.uint8)

In [15]:
# indices,distances=gmu.compute_nearest_neighbors(np.asarray(pcdNode.resource.points),np.asarray(identityPcd.points))

In [16]:
# indices=indices.flatten()
# threshold_indices = np.where(distances <= distance_threshold)[0]
# object_scalar[threshold_indices] = objectArray[indices[threshold_indices]]
# class_scalar[threshold_indices] = classArray[indices[threshold_indices]]

In [17]:
# #remap objectArray
# names=np.array([int(n.name) for n in objectNodes])
# object_scalar=names[object_scalar]

In [18]:
# #temp
# temp_pcd=pcdNode.resource.select_by_index(np.where(distances<distance_threshold)[0])
# o3d.visualization.draw_geometries([temp_pcd] + [joined_geometries])

## EXPORT

las file

In [19]:
# gmu.las_add_extra_dimensions(las,(class_scalar,object_scalar),['classes','objects'],['uint8','uint8'])
# print(list(las.point_format.dimension_names))

In [20]:
# las.write(output_folder/f'{pcdNode.name}_pcd.laz')
# print(f'{pcdNode.name}_pcd.laz')

save graph

In [21]:
for name in np.unique(np.array([n.derivedFrom for n in objectNodes])):
    nodes=[n for n in objectNodes if n.derivedFrom==name]
    tl.nodes_to_graph(nodes,
                    graphPath=str(output_folder/f'{name}_graph.ttl'),
                    save=True)

save objectNodes geometries

In [22]:
for name in np.unique(np.array([n.derivedFrom for n in objectNodes])):
    nodes=[n for n in objectNodes if n.derivedFrom==name]
    t0.write_obj_with_submeshes(output_folder/f'{name}_objects.obj', [n.resource for n in nodes], [n.name for n in nodes])
    print(f'{name} written!')

05_MedOffice_01_F2 written!
06_MedOffice_02_B1 written!
06_MedOffice_02_F1 written!
06_MedOffice_02_F2 written!
06_MedOffice_02_F3 written!
07_MedOffice_03_F3 written!
07_MedOffice_03_F4 written!
07_MedOffice_03_F5 written!
16_Facility_01_F1 written!
19_MedOffice_07_F4 written!
32_ShortOffice_05_F1 written!
32_ShortOffice_05_F2 written!
32_ShortOffice_05_F3 written!
33_SmallBuilding_03_F1 written!
35_Lab_02_F1 written!
35_Lab_02_F2 written!


## FINISH

In [23]:
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

print("runtime: --- %s seconds ---" % (time.time() - start_time))


Current Time = 00:46:39
runtime: --- 9340.315954208374 seconds ---
